# Table of Contents:
* [I. Import libraries](#I.-Import-libraries)
* [II. Data Wangling](#II.-Data-Wangling)
    * [Prepare the dataframe for business](#Prepare-the-dataframe-for-business)
    * [Prepare the dataframe for users](#Prepare-the-dataframe-for-users)
    * [Prepare the dataframe for reviews](#Prepare-the-dataframe-for-reviews)
    * [Prepare categories of businesses](#Prepare-categories-of-businesses)
    * [Add category labels to business dataframe](#Add-category-labels-to-business-dataframe)
    * [Prepare review dataframe with only selected users](#Prepare-review-dataframe-with-only-selected-users)


# I. Import libraries

In [41]:
import json
import math
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# II. Data Wangling
### Prepare the dataframe for business

The original business file is a `csv` file of 148 MB. However, only some attributes are useful. So useless columns are removed from the data. The new dataframe is stored as a csv named df_user.csv. This file has 46.8 MB.

In [42]:
# Read original dataframe
df_business_ori = pd.read_json("./data/yelp_academic_dataset_business.json", lines = True)

In [43]:
# Keep only relevant colunms
df_business = df_business_ori[['business_id','state','latitude','longitude', 'categories','stars']].copy()
# Show an example
df_business[:3]

,business_id,state,latitude,longitude,categories,stars
0,f9NumwFMBDn751xgFiRbNA,NC,35.462724,-80.852612,"Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...",3.5
1,Yzvjg0SayhoZgCljUJRF9Q,AZ,33.569404,-111.890264,"Health & Medical, Fitness & Instruction, Yoga,...",5.0
2,XNoUzKckATkOD1hP6vghZg,QC,45.479984,-73.580070,"Pets, Pet Services, Pet Groomers",5.0


In [22]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_business.to_csv('./data/df_business.csv',index=False)

### Prepare the dataframe for users
The original user file is a `json` file of 3.3 GB. However, we only use attributes `user_id` and `firends`. So a new dataframe for users is created and stored as a `csv` named `df_user.csv`. This file has 2.5 GB.

In [ ]:
# Read large json file of user
with open("./data/yelp_academic_dataset_user.json") as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

In [45]:
# Convert to DataFrame
df_user_ori = pd.DataFrame(data)    
# Keep only relevant colunms
df_users = df_user_ori[['user_id','friends']].copy()

In [ ]:
# Show an example
df_users[:3]

In [11]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_users.to_csv('./data/df_user.csv',index=False)

### Prepare the dataframe for reviews

The original review file is a `json` file of 6.3 GB containing many useless attributes. We have used `jq`, a tool which allows us to select the desired attribute. The result is stored in a `csv` file named `df_reviews.csv`. This file has 729.9 MB.

In [40]:
# Read the review file
df_review = pd.read_csv("./data/df_reviews.csv")
# Show an example
df_review.head()

,review_id,user_id,business_id,stars,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,2017-01-14 21:56:57


### Prepare categories of businesses

In [23]:
# Read prepared csv mapping category name to category label
df_categories = pd.read_csv('name_category.csv')

In [24]:
def correction_name(x):
    """
        Sanitize category name
        x: row of the dataframe
        Return: sanitized category name
    """
    name = x[0]
    name = name.replace("'", "")
    return name

def correction_category(x):
    """
        Sanitize category label
        x: row of the dataframe
        Return: sanitized category
    """
    category = x[1]
    category = category.replace(" ", "")
    category = category.replace("Consruction", "Construction")
    category = category.replace("Constructor", "Construction")
    category = category.replace("Entertainement", "Entertainment")
    category = category.replace("Entertainemnt", "Entertainment")
    category = category.replace("Entertainments", "Entertainment")
    category = category.replace("Merchants", "Merchant")
    category = category.replace("'", "")
    category = category.replace("Mischellaneous", "Miscellaneous")
    category = category.replace("food", "Food")
    return category

In [27]:
# Sanitize category name and category label
df_categories['name'] = df_categories.apply(lambda x: correction_name(x), axis=1)
df_categories['category'] = df_categories.apply(lambda x: correction_category(x), axis=1)

In [28]:
# Save sanitized file
df_categories.to_csv('name_category.csv', index=False)

### Add category labels to business dataframe

In [29]:
# Add category labels to business dataframe
df_business['categories'] = df_business.categories.apply(lambda x: [] if x == None else x.split(', '))

In [31]:
# Create a dictionary mapping category name to category label
dict_categories = df_categories.set_index('name')['category'].to_dict()
# Tokenize the category label and store in a dictionary
indexCategory = {val: key for key, val in enumerate(set(df_categories['category']))}

In [32]:
def create_category_mask(row): 
    """
        For categories of each business, create a mask of categories
        row: row of the dataframe
        Return: mask of categories. If 1 at index i, it means that the business have i-th category.
    """
    res  = np.zeros(numberCat)
    for r in row:
        category = dict_categories.get(r)
        index = indexCategory.get(category)
        res[index] = 1
    return res > 0
# Vectorize the function 
#create_category_mask_vect = np.vectorize(test, signature='()->(k)')

In [33]:
# Fetch the number of categories
numberCat = len(set(df_categories['category']))
# Create category mask for each business in reviews
categoryMask = df_business.apply(lambda row: create_category_mask(row.categories),axis=1)

In [35]:
# For each category, add a new column to the business dataframe
for category_name in set(df_categories['category']):
    # Create "category_name" column, and assign the category mask
    # If mask at row i is True, it means that the business at row i has category: "category_name"
    df_business[category_name] = [tup[indexCategory.get(category_name)] for tup in categoryMask]

In [37]:
# Show an example
df_business.head(2)

,business_id,state,latitude,longitude,categories,stars,Travel,BusinessSupport,Home,Sport,...,RealEstate,Manufacturing,Health,Construction,Automotive,Computers,Entertainment,PersonalCare,Miscellaneous,Food
0,f9NumwFMBDn751xgFiRbNA,NC,35.462724,-80.852612,"[Active Life, Gun/Rifle Ranges, Guns & Ammo, S...",3.5,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
1,Yzvjg0SayhoZgCljUJRF9Q,AZ,33.569404,-111.890264,"[Health & Medical, Fitness & Instruction, Yoga...",5.0,False,False,False,True,...,False,False,True,False,False,False,False,True,False,False


In [38]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_business.to_csv('./data/df_business.csv',index=False)

### Prepare review dataframe with only selected users
For our study, we need to infer user's home, so we need **at least 3 reviews**. Also, we will study users with **at least 3 friends**. 

In [ ]:
# Convert the string of friends to a list
df_users['friend_list'] = df_users.apply(lambda x: x.friends.split(', '), axis=1)
df_users = df_users.drop('friends', axis=1)

In [ ]:
# We count the number of reviewe per user
df_numberOfReviews = df_reviews.groupby('user_id').count().drop(['business_id', 'stars', 'date'], axis = 1).rename(columns={'review_id': 'review_count'})

In [ ]:
# We merge the df_users dataset to the df_numberOfReviews dataset
df_users = pd.merge(df_users, df_numberOfReviews, left_on='user_id', right_on='user_id', how='inner')

In [ ]:
# We create a mask for users with more than 3 friends
mask = df_users.apply(lambda x: len(x.friend_list) >= 3, axis=1)

In [ ]:
print('Number of users with more than 3 reviews:', df_numberOfReviews[df_numberOfReviews['review_count'] >= 3].shape[0])
print('Number of users with more than 3 friends:', df_users[mask].shape[0])

In [ ]:
# Select users that satisfy both conditions
df_selectedUsers = df_users[mask&(df_users['review_count'] >= 3)]
print('Number of users with more than 4 reviews and with more than 3 friends:', df_selectedUsers.shape[0])

In [ ]:
# Number of checkins of the selected users
print('Number of checkins of selected users:', df_selectedUsers['review_count'].sum())

Prepare a dataframe containly only selected users.

In [ ]:
# Merge review and business dataframe
df_reviews = pd.merge(df_reviews, df_business, left_on='business_id', right_on='business_id', how='inner').drop(['state'], axis=1)

In [ ]:
# Select only rows of selected users
df_selectedReviews = df_reviews[df_reviews["user_id"].isin(df_selectedUsers.user_id)]

In [ ]:
# Convert date to datetime object
df_selectedReviews["date"] = pd.to_datetime(df_selectedReviews.date.values)

In [ ]:
# Show an example
df_selectedReviews.head()

In [ ]:
# Save smaller dataframe as csv in order to not load the entire dataset next time
df_selectedReviews.to_csv('./data/selectedReviews.csv',index=False)